In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Reading Excel data

df_foodsales = pd.read_excel('de_challenge_data.xlsx', sheet_name='FoodSales', skiprows=1, usecols='A:I')
df_foodsales

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
0,ID07351,2022-01-01 00:00:00,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04 00:00:00,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07 00:00:00,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,2022-01-10 00:00:00,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
4,ID07355,2022-01-13 00:00:00,East,Boston,Cookies,Arrowroot,38,2.18,82.84
...,...,...,...,...,...,...,...,...,...
242,ID07590,2023-12-18 00:00:00,East,Boston,Cookies,Arrowroot,34,2.18,74.12
243,ID07591,2023-12-21 00:00:00,East,Boston,Cookies,Chocolate Chip,245,1.87,458.15
244,ID07592,2023-12-24 00:00:00,East,Boston,Crackers,Whole Wheat,30,3.49,104.7
245,ID07593,2023-12-27 00:00:00,West,Los Angeles,Bars,Bran,30,1.87,56.1


In [3]:
# Rename columns in case column name is not the same

df_foodsales.columns = ['id', 'date', 'region', 'city', 'category', 'product', 'qty', 'unitprice', 'totalprice']
df_foodsales

,id,date,region,city,category,product,qty,unitprice,totalprice
0,ID07351,2022-01-01 00:00:00,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04 00:00:00,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07 00:00:00,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,2022-01-10 00:00:00,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
4,ID07355,2022-01-13 00:00:00,East,Boston,Cookies,Arrowroot,38,2.18,82.84
...,...,...,...,...,...,...,...,...,...
242,ID07590,2023-12-18 00:00:00,East,Boston,Cookies,Arrowroot,34,2.18,74.12
243,ID07591,2023-12-21 00:00:00,East,Boston,Cookies,Chocolate Chip,245,1.87,458.15
244,ID07592,2023-12-24 00:00:00,East,Boston,Crackers,Whole Wheat,30,3.49,104.7
245,ID07593,2023-12-27 00:00:00,West,Los Angeles,Bars,Bran,30,1.87,56.1


## Cases that we should drop
- merge call
- actual values = column name

In [4]:
# Data in case of blank data and merge cell in de_challenge_data.xlsx
df_foodsales[df_foodsales.isnull().any(axis=1)]

,id,date,region,city,category,product,qty,unitprice,totalprice
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Data that is column name
df_foodsales[df_foodsales['id']=='ID']

,id,date,region,city,category,product,qty,unitprice,totalprice
124,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice


## Drop these cases

In [6]:
df_foodsales_wo_nan = df_foodsales.dropna(subset=['date'])
df_foodsales_wo_nan

,id,date,region,city,category,product,qty,unitprice,totalprice
0,ID07351,2022-01-01 00:00:00,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04 00:00:00,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07 00:00:00,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,2022-01-10 00:00:00,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
4,ID07355,2022-01-13 00:00:00,East,Boston,Cookies,Arrowroot,38,2.18,82.84
...,...,...,...,...,...,...,...,...,...
242,ID07590,2023-12-18 00:00:00,East,Boston,Cookies,Arrowroot,34,2.18,74.12
243,ID07591,2023-12-21 00:00:00,East,Boston,Cookies,Chocolate Chip,245,1.87,458.15
244,ID07592,2023-12-24 00:00:00,East,Boston,Crackers,Whole Wheat,30,3.49,104.7
245,ID07593,2023-12-27 00:00:00,West,Los Angeles,Bars,Bran,30,1.87,56.1


In [7]:
df_foodsales_wo_nan_col = df_foodsales_wo_nan[df_foodsales_wo_nan['id']!='ID']
df_foodsales_wo_nan_col

,id,date,region,city,category,product,qty,unitprice,totalprice
0,ID07351,2022-01-01 00:00:00,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,2022-01-04 00:00:00,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,2022-01-07 00:00:00,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,2022-01-10 00:00:00,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
4,ID07355,2022-01-13 00:00:00,East,Boston,Cookies,Arrowroot,38,2.18,82.84
...,...,...,...,...,...,...,...,...,...
242,ID07590,2023-12-18 00:00:00,East,Boston,Cookies,Arrowroot,34,2.18,74.12
243,ID07591,2023-12-21 00:00:00,East,Boston,Cookies,Chocolate Chip,245,1.87,458.15
244,ID07592,2023-12-24 00:00:00,East,Boston,Crackers,Whole Wheat,30,3.49,104.7
245,ID07593,2023-12-27 00:00:00,West,Los Angeles,Bars,Bran,30,1.87,56.1
